### makemore: lect01

#### 20240227-061500

In [14]:
words = open("names.txt", "r").read().splitlines()

In [15]:
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [16]:
b = {}
for w in words[:3]:
    print("-" * 80)
    chs = ["<S>"] + list(w) + ["E"]
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) + 1
        print(ch1, ch2)

--------------------------------------------------------------------------------
<S> e
e m
m m
m a
a E
--------------------------------------------------------------------------------
<S> o
o l
l i
i v
v i
i a
a E
--------------------------------------------------------------------------------
<S> a
a v
v a
a E


In [17]:
b

{('<S>', 'e'): 1,
 ('e', 'm'): 1,
 ('m', 'm'): 1,
 ('m', 'a'): 1,
 ('a', 'E'): 3,
 ('<S>', 'o'): 1,
 ('o', 'l'): 1,
 ('l', 'i'): 1,
 ('i', 'v'): 1,
 ('v', 'i'): 1,
 ('i', 'a'): 1,
 ('<S>', 'a'): 1,
 ('a', 'v'): 1,
 ('v', 'a'): 1}

get all counts 

In [18]:
b = {}
for w in words:
    chs = ["<S>"] + list(w) + ["E"]
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) + 1

In [19]:
sorted(b.items(), key=lambda u: -u[1])[:10]

[(('n', 'E'), 6763),
 (('a', 'E'), 6640),
 (('a', 'n'), 5438),
 (('<S>', 'a'), 4410),
 (('e', 'E'), 3983),
 (('a', 'r'), 3264),
 (('e', 'l'), 3248),
 (('r', 'i'), 3033),
 (('n', 'a'), 2977),
 (('<S>', 'k'), 2963)]

In [2]:
import torch

In [4]:
a = torch.zeros((3, 5))

In [5]:
a

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [7]:
a.dtype

torch.float32

In [8]:
a = torch.zeros((3, 5), dtype=torch.int32)

In [9]:
a

tensor([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]], dtype=torch.int32)

In [11]:
a[1, 3] = 1

In [12]:
a

tensor([[0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0]], dtype=torch.int32)